# Maximum Likelihood

Suppose we have a collection of independent random samples of random variables $X_1, X_2, ..., X_n$ from the same probability distribution, each with an associated PDF $p_\theta(x)$ which depends on some parameter $\theta$. The joint PDF is $p_\theta(x_1, x_2, ..., x_n) = p_\theta(x_1)p_\theta(x_2)...p_\theta(x_n)$. 

$L(\theta) = p_\theta(x_1, x_2, ..., x_n)$

This is called the **likelihood function**. The Maximum Likelihood Estimator of $\theta$ (MLE) is the value $\hat{\theta}$ that maximizes the likelihood. 

---
##### Some Useful Logarithmic Properties

* **Product Rule** $\ln{(AB)} = \ln{(A)} + \ln{(B)}$
* **Quotient Rule** $\ln{(\frac{A}{B})} = \ln{(A)} - \ln{(B)}$
* **Power Rule** $\ln{(A^x)} = x\ln{(A)}$

---
#### Examples

1. (Single parameter optimization) We have a collection of random samples that are binomially distributed with probability 0.5. Let's pretend we don't know that they have probability $p=0.5$ and want to estimate said parameter. The likelihood function for the binomial distribution can be given by the following: $L(p) = p^x(1-p)^{n-x}$, where $x$ is the total number of successes and $n$ is the total number of samples. 

**Using `optim()`:**

syntax: `optim(par, fn)`

arguments: 
* `par` Initial values for the parameters to be optimized over.
* `fn` A function to be minimized, with the first argument being the vector of parameters over which minimization is to take place. It should return a scalar result.

output:
* `par` The MLE set of parameters found.
* `value` The value of `fn` corresponding to `par`.
* `counts` The first value in the vector gives the number of calls to `fn`.

In [ ]:
# draw 10 binomially-distributed samples with p=0.5

data1 <- rbinom(10, 1, 0.5)
x <- sum(data1) # number of heads
n <- length(data1) # number of samples
data1

In [ ]:
# make likelihood function

binom_like <- function(p){
    L <- (p^x)*((1-p)^(n-x))
    return(L)
}

In [ ]:
# plot the likelihood function for varying values of p

seq_p <- seq(0,1, by=0.01) # create a sequence of numbers from 0 to 1 with breaks of size 0.01
seq_y <- c()
for(s in seq_p){
  seq_y <- append(seq_y, binom_like(s)) # calculate the likelihood for every value in seq_p
}

# we can see which value gave us the maximum likelihood
ind <- which(seq_y == max(seq_y))
seq_p[ind]

plot(seq_p, seq_y, xlab = "p", ylab = "likelihood")
abline(v=seq_p[ind], col="red", lty = 2)

In [ ]:
# using the optim() function; **NOTE** need to change the sign of the likelihood function

binom_like <- function(p){
    L <- (p^x)*((1-p)^(n-x))
    return(-L)
}

res1 <- optim(0.9, binom_like)
res1

In [ ]:
## an aside of what changing the sign does
# plot the likelihood function for varying values of p

seq_p <- seq(0,1, by=0.01) # create a sequence of numbers from 0 to 1 with breaks of size 0.01
seq_y <- c()
for(s in seq_p){
  seq_y <- append(seq_y, binom_like(s)) # calculate the likelihood for every value in seq_p
}

plot(seq_p, seq_y)

In [ ]:
# what happens with different starting values? issue of convergence

optim(5, binom_like)

In [ ]:
# draw 1000 binomially-distributed samples with p=0.5

data2 <- rbinom(1000, 1, 0.5)
x <- sum(data2) # number of heads
n <- length(data2) # number of samples
x

In [ ]:
# plot the likelihood function for varying values of p

binom_like <- function(p){
    L <- (p^x)*((1-p)^(n-x))
    return(L)
}


seq_p <- seq(0,1, by=0.01) # create a sequence of numbers from 0 to 1 with breaks of size 0.01
seq_y <- c()
for(s in seq_p){
  seq_y <- append(seq_y, binom_like(s)) # calculate the likelihood for every value in seq_p
}

ind <- which(seq_y == max(seq_y))
seq_p[ind]

plot(seq_p, seq_y)
abline(v = seq_p[ind], col="red", lty = 2)

In [ ]:
# using the optim() function; **NOTE** need to change the sign of the likelihood function
# with more samples, because the likelihood is a product, it becomes nearly impossible to actually maximize

binom_like <- function(p){
    L <- (p^x)*((1-p)^(n-x))
    return(-L)
}

optim(0.9, binom_like) #even if we change starting value


In [ ]:
# create log-likelihood function
# always want to take the log of the individual expression components and not the original likelihood function - it
# preserves accuracy that would be lost from rounding

binom_logLike <- function(p){
    logL <- x*log(p)+(n-x)*log(1-p)
    return(-logL)
}

res2 <- optim(0.9, binom_logLike)
res2



In [ ]:
# changing to a log-scale doesn't affect the MLE

seq_p <- seq(0,1, by=0.01) # create a sequence of numbers from 0 to 1 with breaks of size 0.01
seq_z <- c()
for(s in seq_p){
  seq_z <- append(seq_z, -binom_logLike(s)) # calculate the likelihood for every value in seq_p
}

plot(seq_p, seq_z)
ind <- which(seq_z == max(seq_z))
seq_z[ind]


In [ ]:
# what about the log-likelihood on the original dataset of 20 samples?
x <- sum(data1)
n <- length(data1)

optim(0.9, binom_logLike)

In [ ]:
res1

#### Examples

2. (Multiple parameter optimization) Suppose we have 10 random samples drawn from a normal distribution, with unknown mean $\mu$ and variance $\sigma^2$. The log-likelihood function for the normal distribution is $L(\mu,\sigma^2) = -.5n\ln{(2\pi)}-.5n\ln{(\sigma^2)}-\frac{1}{2\sigma^2}\sum_i(y_i-\mu)^2$. Find the MLEs for both $\mu$ and $\sigma^2$.

In [ ]:
data <- rnorm(10, 50, 2)
n <- length(data)

In [ ]:
# define the log-likelihood function

normal_logLike <- function(theta){
  mu <- theta[1]
  sigma2 <- theta[2]
  
  logL <- -.5*n*log(2*pi) -.5*n*log(sigma2) - (1/(2*sigma2))*sum((data-mu)**2)
  return(-logL)
}

In [ ]:
optim(c(mean(data), var(data)), normal_logLike)


In [ ]:
# contour plot the likelihood function for varying values of a, b
#library(plotly)

mu_seq <- round(seq(min(data), max(data), by=0.01),2)
s2_seq <- seq(var(data)-1, var(data)+1, by=0.01)

grid_like <- matrix(NA, length(mu_seq),length(s2_seq))
for(i in 1:length(mu_seq)){
  for(j in 1:length(s2_seq)){
    grid_like[i,j] <- normal_logLike(c(mu_seq[i], s2_seq[j]))
  }
}

head(grid_like)

In [ ]:
library(plotly)
fig <- plot_ly(z = grid_like, type = 'contour', x = s2_seq, y = mu_seq,colorscale = 'Viridis') #note the axes!

fig

### Questions

1. Suppose the weights of randomly selected American female college students are normally distributed with unknown mean  and standard deviation . A random sample of 10 American female college students yielded the following weights (in pounds):

115 122 130 127 149 160 152 138 149 180

Using the given sample, find a maximum likelihood estimate of $\mu$ and $\sigma^2$.

2. Suppose that the lifetime of Badger brand light bulbs is modeled by an exponential distribution with (unknown) parameter $\lambda$. We test 5 bulbs and find they have lifetimes of 2, 3, 1, 3, and 4 years, respectively. What is the MLE for $\lambda$?

Log likelihood function for exponential:
$nln(\lambda)-\lambda\sum_{j=1}^nx_j$

## Assignment

According to the Jukes and Cantor 1969 (JC69) model, the probability that two nucleotides are identical to each other is:

$\frac{1}{4}+\frac{3}{4}e^{-4\lambda/3}$

and the probability that two nucleotides differ from each other is:

$\frac{1}{4}-\frac{1}{4}e^{-4\lambda/3}$

So the likelihood function, assuming independence among sites, if $d$ out of $S$ nucleotides differ between the two sequences is:
$L(\lambda) = \left(\frac{1}{4}+\frac{3}{4}e^{-4\lambda/3}\right)^{S-d}\left(\frac{1}{4}-\frac{1}{4}e^{-4\lambda/3}\right)^d$

1. Read in the two sequences using the function provided and calculate the number of pairwise differences $d$ using a loop.

In [ ]:
x <- read.table('inputs/hw_09_data.txt',sep = '\t')
seq1 <- strsplit(as.vector(x$V1[2]),split = '')[[1]]
seq2 <- strsplit(as.vector(x$V1[3]),split = '')[[1]]

S <- 0
d <- 0

2. Implement a likelihood function based on the JC69 model. The likelihood function is parameterized in terms of $\lambda$ which is the genetic distance. $\lambda$ is proportional to the mutation rate and the amount of time the two species have been diverging from each other. The type of genetic distance is fundamental in many phylogenetic studies and other studies comparing DNA sequences.

    Plot the log-likelihood function as a function of varying $\lambda$ values from 0 to 1.  


3. Create an R function that calculates the logarithm of the likelihood. This function requires $\lambda$ as an argument. Plot the log-likelihood function as a function of the same varying $\lambda$ values from 0 to 1. Did changing the scale to a log-scale affect the MLE?

4. With a starting value of 0.5, use the `optim()` function in R to optimize the log-likelihood function for $\lambda$. Does this value match the one observed in the plot?

5. Transitions (changes between A and G and between C and T) tend to occur at a higher rate than transversions (all other changes). Calculate the number of transitions $ds$ and the number of transversions $dv$ from the two sequences. The Kimura two-parameter (K2P) model is specified in terms of two parameters $\alpha$ and $\beta$. These two parameters can be interpreted as the rate of transitions and transversions per unit time, respectively, multiplied by the total amount of time. The likelihood function under this model is:
$L(\alpha, \beta) = \left(\frac{1}{4}+\frac{1}{4}e^{-4\beta}+\frac{1}{2}e^{-2(\alpha+\beta)}\right)^{S-dv-ds}\left(\frac{1}{4}+\frac{1}{4}e^{-4\beta}-\frac{1}{2}e^{-2(\alpha+\beta)}\right)^{ds}\left(\frac{1}{4}-\frac{1}{4}e^{-4\beta}\right)^{dv}$.

    Make a function in R that calculates the logarithm of this likelihood. It should take a single vector $\theta$, the first element corresponding to $\alpha$, and the second $\beta$. 

6. Calculate the number of transitions and transversions between the two sequences using some lines of code (`if` statements will be useful - remember `&` is used for 'and' statements, `|` is used for 'or') (Not sure you got all the conditions rights? Hint: what should the transitions and transversions sum up to?). Plot the log-likelihood function for varying $\alpha$ and $\beta$ values in a contour plot. Approximately where is the maximum likelihood estimate?

In [ ]:
ds <- 0
dv <- 0


In [ ]:
library(plotly)
a <- seq(0, 1, by=0.01)
a <- a[-1] # why do I take out the first element? the log-likelihood function will return -Inf for a = b = 0
b <- a

grid_like <- matrix(NA, length(a),length(b))
# fill in the likelihood grid, similar to how we did in lab

# make the contour plot - pay attention to the axes!

7. With starting values of 0.5 for both $\alpha$ and $\beta$, use the `optim()` function in R to optimize this log-likelihood function for $\alpha$ and $\beta$. Does this value match the one observed in the plot?